# 目的
`Tensorflow`を用いた深層学習を試す。   
『Pythonによるスクレイピング＆機械学習』の５章に沿って、Tensorflowの使い方を学習する。

## インポート

In [1]:
import tensorflow as tf

# Tensorflowでの変数表現

## 定数の定義
`tf.constant()`を利用する

In [ ]:
a = tf.constant(120, name="a")
b = tf.constant(130, name="b")
c = tf.constant(140, name="c")
print(a, b, c)

## 変数の定義
`tf.Variable`を利用する

In [ ]:
v = tf.Variable(0, name="v")
print(v)

## データフローグラフの定義
`tf.assign`で処理を定義する

In [ ]:
for line in tf.assign.__doc__.split("\n"):
    print(line)

In [ ]:
calc_op = a + b + c
assign_op = tf.assign(v, calc_op)
print(calc_op, assign_op)

## セッションの開始と処理の実行
`tf.Session()`でセッションを初期化し、`Session.run(op)`で`op`を実行する

In [ ]:
sess = tf.Session()
# print(sess.run(calc_op))
sess.run(assign_op)
print(sess.run(v))

同じ名前の変数ないし定数を作成しようとしても、自動的に番号が割り振られる（"v_1"のように）ことに注意する。

# プレースホルダ
テンプレートに値を当てはめるための仕組み。`tf.placeholder`を利用する

In [ ]:
# 整数型、要素３この配列を定義
a = tf.placeholder(tf.int32, [3])

# 配列中の各要素を２倍にする演算を定義
b = tf.constant(2)
x2_op = a * b

In [ ]:
# セッションの開始
sess = tf.Session()

# プレースホルダに値を当てはめて実行
r1 = sess.run(x2_op, feed_dict={a:[1,2,3]})
r2 = sess.run(x2_op, feed_dict={a:[10,20,30]})
r1, r2

## 配列のサイズを決めない方法
Noneを入れればよい

In [ ]:
# 整数型、要素３この配列を定義
a = tf.placeholder(tf.int32, [None])

# 配列中の各要素を２倍にする演算を定義
b = tf.constant(2)

x2_op = a * b

In [ ]:
# セッションの開始
sess = tf.Session()

# プレースホルダに値を当てはめて実行
r1 = sess.run(x2_op, feed_dict={a:[1,2,3,3,5]})
r2 = sess.run(x2_op, feed_dict={a:[10,20,30]})
r1, r2

# 機械学習に挑戦
BMIのデータで練習

## データの読出し

In [9]:
# 身長,体重,ラベルのCSVデータを読み出す --- (※1)
csv = pd.read_csv("./data/bmi.csv")
# データを正規化 --- (※2)
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100

## ラベルの変換
ラベルを３次元のクラスで表す。   
別に１次元でいいのでは？という話もあるが、なにか理由があった気がする。

In [10]:
# ラベルを三次元のクラスで表す --- (※3)
# - thin=(1,0,0) / normal=(0,1,0) / fat=(0,0,1)
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

## テストデータの用意

In [11]:
# 正解率を求めるためにテストデータを準備 --- (※4)
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

## データフローの構築など

In [12]:
# データフローグラフを構築する --- (※5)
# データを入れるプレースホルダを宣言
x  = tf.placeholder(tf.float32, [None, 2]) # 身長,体重のデータを入れる
y_ = tf.placeholder(tf.float32, [None, 3]) # 答えのラベルを入れる

# 変数を宣言 --- (※6)
W = tf.Variable(tf.zeros([2, 3])); # 重み
b = tf.Variable(tf.zeros([3])); # バイアス

## モデルの用意
ソフトマックス回帰と交叉エントロピーを利用する

In [13]:
# ソフトマックス回帰を定義 --- (※7)
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 訓練するモデルの用意 --- (※8)
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 正解率を求める
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

In [14]:
y,cross_entropy, optimizer, train, predict, accuracy

(<tf.Tensor 'Softmax:0' shape=(?, 3) dtype=float32>,
 <tf.Tensor 'Neg:0' shape=() dtype=float32>,
 <tf.Operation 'GradientDescent' type=NoOp>,
 <tf.Tensor 'Equal:0' shape=(?,) dtype=bool>,
 <tf.Tensor 'Mean:0' shape=() dtype=float32>)

## モデルの訓練
ここまではあくまでtensorflowのセッションで用いる変数の定義。   
実際の訓練はセッションを開始してから行う。

In [15]:
# セッションを開始
sess = tf.Session()
sess.run(tf.global_variables_initializer()) #変数を初期化
# テストデータを用いて学習させる
size = 100
for step in range(3500):
    i = (step * size) % 14000
    rows = csv[1 + i : 1 + i + size]
    x_pat = rows[["weight","height"]]
    y_ans = list(rows["label_pat"])
    fd = {x: x_pat, y_: y_ans}
    sess.run(train, feed_dict=fd)
    if step % 500 == 0:
        cre = sess.run(cross_entropy, feed_dict=fd)
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
        print("step=", step, "cre=", cre, "acc=", acc)

step= 0 cre= 108.663 acc= 0.3242
step= 500 cre= 57.5887 acc= 0.8904
step= 1000 cre= 45.0209 acc= 0.898
step= 1500 cre= 41.6543 acc= 0.9566
step= 2000 cre= 34.664 acc= 0.943
step= 2500 cre= 34.287 acc= 0.9674
step= 3000 cre= 26.8808 acc= 0.9726


## 正解率を求める

In [16]:
# 最終的な正解率を求める
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("正解率=", acc)

正解率= 0.9712


## 訓練されたモデルの保存
`tf.train.Saver()`を用いる。   
生成されるファイルは４つ（`*.data-~`, `*.index`, `*.meta`, `checkpoint`）

In [17]:
saver = tf.train.Saver()

In [18]:
saver.save(sess, save_path="./data/model/model.ckpt")

'./data/model/model.ckpt'

## モデルの読み込み
`saver.restore(session ,model_path)`を利用する

In [19]:
sess = tf.Session()
saver.restore(sess, "./data/model/model.ckpt")

INFO:tensorflow:Restoring parameters from ./data/model/model.ckpt


In [ ]:
acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print("正解率=", acc)

## 所感
正直ややこしいというのが感想。要点をまとめると、   

* 重みとバイアスが最適化したい変数であるから`tf.Variable`で定義する
* 訓練データと正解ラベルは定数として扱うが、計算で順次与えるものだから`tf.placeholder`を用いて入れ物を定義
* 回帰に用いる関数（ソフトマックス関数など）、最小化したい関数（交叉エントロピーなど）、訓練用関数を定義する
* あとはtf.Session内で次々と訓練データと正解ラベルを与えて訓練する

# TensorBoardの利用
`tf.train.SummaryWriter(name, graph=sess.graph)`で利用できる

## 掛け算のコードの視覚化

In [2]:
# データフローグラフの構築
a = tf.constant(20, name="a")
b = tf.constant(30, name="b")
mul_op = a * b

# セッションの生成
sess = tf.Session()

# TensorBoardの利用
tw = tf.summary.FileWriter("log_dir", graph=sess.graph)

sess.run(mul_op)

600

## BMIのTensorboardの作成

In [6]:
# 身長,体重,ラベルのCSVデータを読み出す --- (※1)
csv = pd.read_csv("./data/bmi.csv")
# データを正規化 --- (※2)
csv["height"] = csv["height"] / 200
csv["weight"] = csv["weight"] / 100
# ラベルを三次元のクラスで表す --- (※3)
# - thin=(1,0,0) / normal=(0,1,0) / fat=(0,0,1)
bclass = {"thin": [1,0,0], "normal": [0,1,0], "fat": [0,0,1]}
csv["label_pat"] = csv["label"].apply(lambda x : np.array(bclass[x]))

# 正解率を求めるためにテストデータを準備 --- (※4)
test_csv = csv[15000:20000]
test_pat = test_csv[["weight","height"]]
test_ans = list(test_csv["label_pat"])

# データフローグラフを構築する --- (※5)
# データを入れるプレースホルダを宣言
x  = tf.placeholder(tf.float32, [None, 2], name="x") 
y_ = tf.placeholder(tf.float32, [None, 3], name="y_") 

# 変数を宣言 --- (※6)
with tf.name_scope('interface') as scope:
    W = tf.Variable(tf.zeros([2, 3]), name="W"); # 重み
    b = tf.Variable(tf.zeros([3]), name="b"); # バイアス
    # ソフトマックス回帰を定義 --- (※7)
    with tf.name_scope('softmax') as scope:
        y = tf.nn.softmax(tf.matmul(x, W) + b)

# モデルを訓練する --- (※8)
with tf.name_scope('loss') as scope:
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

with tf.name_scope('training') as scope:
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(cross_entropy)

# 正解率を求める
with tf.name_scope('accuracy') as scope:
    predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

# セッションを開始
with tf.Session() as sess:
    tw = tf.summary.FileWriter("log_dir", graph=sess.graph)
    sess.run(tf.global_variables_initializer()) #変数を初期化
    # テストデータを用いて学習させる
    for step in range(3500):
        i = (step * 100) % 14000
        rows = csv[1 + i : 1 + i + 100]
        x_pat = rows[["weight","height"]]
        y_ans = list(rows["label_pat"])
        fd = {x: x_pat, y_: y_ans}
        sess.run(train, feed_dict=fd)
        if step % 500 == 0:
            cre = sess.run(cross_entropy, feed_dict=fd)
            acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
            print("step=", step, "cre=", cre, "acc=", acc)

    # 最終的な正解率を求める
    acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
    print("正解率=", acc)

step= 0 cre= 108.663 acc= 0.3242
step= 500 cre= 57.5887 acc= 0.8904
step= 1000 cre= 45.0209 acc= 0.898
step= 1500 cre= 41.6543 acc= 0.9566
step= 2000 cre= 34.664 acc= 0.943
step= 2500 cre= 34.287 acc= 0.9674
step= 3000 cre= 26.8808 acc= 0.9726
正解率= 0.9712
